In [1]:
import itertools as _itertools
from math import ceil
from sage.all import *

def diameter_D(sigmadual):
    rays = [vector(r) for r in list(sigmadual.rays())]
    n = len(rays)
    if n == 0:
        return 0
    combos = []
    for coeffs in _itertools.product([0, 1], repeat=n):
        v = sum(coeffs[i] * rays[i] for i in range(n))
        combos.append(v)
    max_dist = 0.0
    for i in range(len(combos)):
        for j in range(i + 1, len(combos)):
            d = (combos[i] - combos[j]).norm()
            if d > max_dist:
                max_dist = float(d)
    return int(ceil(max_dist))

def minimal_face_containing_w(sigma, w):
    dim_sigma = sigma.dimension()
    for k in range(1, dim_sigma + 1):
        for F in sigma.faces(k):
            if F.contains(w):
                return F
    return None

def compute_weight(vi, w, D, cone_vectors):
    from itertools import product
    n = len(vi)
    minus_vi = -vi
    delta = Cone(cone_vectors)
    deltaDual = delta.dual()
    Hw = [vector(h) for h in deltaDual.Hilbert_basis()]
    H_non_orth = [h for h in Hw if w.dot_product(h) != 0]
    if not H_non_orth:
        return matrix(QQ, n, 0)
    vals_vi = [minus_vi.dot_product(h) for h in H_non_orth]
    c = min(vals_vi)
    valid_points = []
    for coords in product(range(-D, D + 1), repeat=n):
        u = vector(coords)
        if u.is_zero() or not deltaDual.contains(u):
            continue
        vali = minus_vi.dot_product(u)
        valw = w.dot_product(u)
        if vali < c and valw == 0:
            valid_points.append(u)
    if not valid_points:
        return matrix(QQ, n, 0)
    Critiw = Matrix(QQ, n, len(valid_points), lambda i, j: valid_points[j][i])
    return Critiw

def criticalArrow(vecsList, w):
    sigma = Cone(vecsList)
    D = diameter_D(sigma.dual())
    mu = minimal_face_containing_w(sigma, w)
    if mu is None:
        return 0
    mu_dim = mu.dimension()
    mu_rays = [vector(r) for r in mu.rays()]
    Critw = matrix(QQ, len(vecsList[0]), 0)
    for vi in mu_rays:
        other_rays = [v for v in vecsList if v != vi]
        Critiw = compute_weight(vi, w, D, other_rays + [-w])
        Critw = Critw.augment(Critiw)
    if Critw.ncols() == 0:
        return 0
    Vw = Critw.column_space()
    mu_ray_matrix = matrix(QQ, mu_rays).transpose()
    mu_orth = mu_ray_matrix.left_kernel()
    dim_arr = Vw.dimension() - Vw.intersection(mu_orth).dimension()
    return mu_dim - dim_arr

In [2]:
myVecsList = [(vector([1,0,0]), vector([a,b,c]), vector([x,y,z]))
              for c in range(0,4) for b in range(0,c+1) for a in range(0,4)
              for z in range(0,4) for y in range(0,z+1) for x in range(0,4)
              if gcd(a,gcd(b,c))==1
              if gcd(x,gcd(y,z))==1
             ]

for vecsList in myVecsList:
    M = matrix(vecsList)
    det = M.determinant()
    if abs(det) <= 1:
        continue
        
    print("generators:", [tuple(v) for v in vecsList])
    sigma = Cone(vecsList)
    hilb = sigma.Hilbert_basis()
    for w in hilb:
        dim_tau = criticalArrow(vecsList, w)
        print(f" w={tuple(w)}, dim_tau={dim_tau}")

generators: [(1, 0, 0), (0, 0, 1), (1, 2, 2)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(1, 2, 2), dim_tau=1
 w=(1, 1, 1), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (3, 2, 2)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(3, 2, 2), dim_tau=1
 w=(2, 1, 1), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (0, 2, 3)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(0, 2, 3), dim_tau=1
 w=(0, 1, 2), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (1, 2, 3)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(1, 2, 3), dim_tau=1
 w=(1, 1, 2), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (2, 2, 3)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(2, 2, 3), dim_tau=1
 w=(1, 1, 2), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (3, 2, 3)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(3, 2, 3), dim_tau=1
 w=(2, 1, 2), dim_tau=1
generators: [(1, 0, 0), (0, 0, 1), (1, 3, 3)]
 w=(1, 0, 0), dim_tau=1
 w=(0, 0, 1), dim_tau=1
 w=(1, 3, 3), dim_tau=1
 w=(1, 1, 1), dim_tau=1
 w=(1,